In [1]:
import sys
import keras.layers

assert sys.version_info >= (3, 7)

In [19]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [21]:
import numpy as np

In [3]:
from pathlib import Path

In [15]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [5]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100, kernel_initializer="lecun_normal",
                                    activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [6]:
dataset = tf.keras.datasets.cifar10.load_data()

In [7]:
optimizer = tf.keras.optimizers.legacy.Nadam(learning_rate=7e-4) #Legacy cause made on a m2
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [8]:
(X_train_full, y_train_full), (X_test, y_test) = dataset

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [9]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [10]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_Mc_Dropout_model.h5",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run__dropout{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [11]:
%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs

Reusing TensorBoard on port 6006 (pid 10642), started 1 day, 20:37:09 ago. (Use '!kill 10642' to kill it.)

In [12]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.9184 - accuracy: 0.3157 - val_loss: 1.8117 - val_accuracy: 0.3796
Epoch 2/100
  33/1407 [..............................] - ETA: 6s - loss: 1.7365 - accuracy: 0.3845

/opt/anaconda3/envs/homl3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 7s 5ms/step - loss: 1.7078 - accuracy: 0.3976 - val_loss: 1.7156 - val_accuracy: 0.3766
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6137 - accuracy: 0.4315 - val_loss: 1.6827 - val_accuracy: 0.4056
Epoch 4/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5552 - accuracy: 0.4514 - val_loss: 1.5739 - val_accuracy: 0.4400
Epoch 5/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5026 - accuracy: 0.4732 - val_loss: 1.6159 - val_accuracy: 0.4334
Epoch 6/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4515 - accuracy: 0.4916 - val_loss: 1.5589 - val_accuracy: 0.4646
Epoch 7/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4158 - accuracy: 0.5064 - val_loss: 1.6073 - val_accuracy: 0.4658
Epoch 8/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3781 - accuracy: 0.5209 - val_loss: 1.5683 - val_accuracy: 0.48

In [13]:
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5284 - accuracy: 0.4934


[1.528371810913086, 0.4934000074863434]

In [16]:
mc_model = tf.keras.Sequential([
    (
        MCAlphaDropout(layer.rate)
        if isinstance(layer, keras.layers.AlphaDropout)
        else layer
    )
    for layer in model.layers
])

In [17]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return Y_probas.argmax(axis=1)

In [22]:
y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = (y_pred == y_valid[:, 0]).mean()
accuracy

157/157 [==============================] - 0s 1ms/step


0.4954